In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

## Create Features

In [ ]:
feature = pd.read_json("train.json")
feature ["num_photos"] = feature ["photos"].apply(len)
feature ["num_features"] = feature ["features"].apply(len)
feature ["num_description_words"] = feature ["description"].apply(lambda x: len(x.split(" ")))
feature ["created"] = pd.to_datetime(feature["created"])
feature ["created_year"] = feature ["created"].dt.year
feature ["created_month"] = feature ["created"].dt.month
feature ["created_day"] = feature ["created"].dt.day

In [ ]:

feature_using = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
                   "num_photos", "num_features", "num_description_words",
                   "created_year", "created_month", "created_day"]
print(feature_using)

In [ ]:
feature['created'].dt.month.value_counts(sort=False).plot(kind='bar')

In [ ]:
new_feature = preprocessing.LabelEncoder()
new_feature.fit(list(feature['manager_id'].values))
feature['manager_id'] = new_feature.transform(list(feature['manager_id'].values))

### Adding manager_id to features

In [ ]:
feature_using.extend(['manager_id'])
rental = feature[feature_using]
print(rental.head())

### Adding manager_skill as feature

In [ ]:
X = feature[feature_using]
Y = feature['interest_level']

temp = pd.concat([X.manager_id, pd.get_dummies(Y)], axis=1).groupby('manager_id').mean()
temp['count'] = X.groupby('manager_id').count().iloc[:,1]

temp['manager_skill'] = temp['high']*2 + temp['medium']

unranked_managers_ixes = temp['count']<20

ranked_managers_ixes = ~unranked_managers_ixes

mean_values = temp.loc[ranked_managers_ixes, ['high','low','medium','manager_skill']].mean()
print(mean_values)
temp.loc[unranked_managers_ixes,['high','low','medium','manager_skill']] = mean_values.values

temp['managers_id'] = temp.index
joined = pd.merge(rental, temp,on='manager_id')
print(joined)

## Show top_five ommon features

In [ ]:
train = pd.read_json("train.json")
all_features = []
for i in train['features']:
    all_features += i
    all_features = [x.strip().lower() for x in all_features]
all_features = sorted(all_features)

In [ ]:
import collections

counter=collections.Counter(all_features)

In [ ]:
common_features = pd.DataFrame(counter.most_common(), columns=['features', 'quantity'])
print(commonFeatures.shape)
common_features = commonFeatures.loc[:40, :]
common_features.plot(kind='bar')

In [ ]:
common_features.head()